In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/methylbenzel/vlspdata/vlsp.csv


# install librari

In [2]:
!pip install "transformers==4.35" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"
     

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 2.6 MB/s eta 0:00:0000:01
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
     

2024-05-14 03:08:15.345513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 03:08:15.345634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 03:08:15.483467: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
df = pd.read_csv("/kaggle/input/d/methylbenzel/vlspdata/vlsp.csv")
df = df[:1000]


In [ ]:
df

In [ ]:
df.describe()

In [ ]:

# Concatenate 'Question' and 'Answer' columns into a new column 'text'
df['text'] = 'question:\n' + df['question'] + '\n\nanswer:\n' + df['answer']

In [ ]:

df.head()

In [ ]:

print(df['text'].loc[0])

In [ ]:

#drop columns other than 'text'
df.drop(columns=['question','answer'], axis=1, inplace=True)

# #drop columns other than 'text'
df.drop(columns=['question','answer'], axis=1, inplace=True)
df = df[:

In [ ]:
train = Dataset.from_pandas(df)

In [ ]:

train

# finetune


In [ ]:

model_id = "genaitraining/llama-2-7b-domain-tuned"

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
#huggingface_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

!mkdir -p ~/.huggingface
!echo -n $huggingface_token > ~/.huggingface/token

In [ ]:
# lets verify
!huggingface-cli whoami

In [ ]:
# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

In [ ]:

%%time
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
#pip install --upgrade peft

In [7]:
pip install git+https://github.com/huggingface/peft

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-0bk5_mew
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-0bk5_mew
  Resolved https://github.com/huggingface/peft to commit 2558dd872dbacd29f6b60668bfee54b594bcbd7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.10.1.dev0-py3-none-any.whl size=249079 sha256=e324d51c8df08cccd769a2acf1b4a0c12f07b8a0041ba3f4d28a6de355e93828
  Stored in directory: /tmp/pip-ephem-wheel-cache-xsc3ze2o/wheels/4c/16/67/1002a2d4daa822eff130e6d85b90051b75d2ce0d26b9448e4a
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0
    Uninstalling peft-0.4.0:
      Successfully uninstalled peft-0.4.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:

%%time
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto")

In [ ]:

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
                          lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM"
                        )

In [ ]:

# Define the training arguments. For full list of arguments, check
#https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
args = TrainingArguments(
    output_dir='llama2-7b-tuned-qna',
    num_train_epochs=1, # adjust based on the data size
    per_device_train_batch_size=2, # use 4 if you have more GPU RAM
    save_strategy="epoch", #steps
    # evaluation_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    seed=42
)

In [ ]:

# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    # eval_dataset=test,
    dataset_text_field='text',
    peft_config=peft_config,
    max_seq_length=1042,
    tokenizer=tokenizer,
    args=args,
    packing=True,
)

In [ ]:


# train
trainer.train()

In [ ]:

# save model in local
trainer.save_model()

In [ ]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
gc.collect()

In [ ]:
%%time
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    'llama2-7b-tuned-qna',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
).cuda()

In [ ]:
print('hello')

In [ ]:
prompt = """
Dựa vào thông tin và trả lời các câu hỏi:
 Trận đánh gây được tiếng vang lúc bấy giờ là trận phục kích của quân ta tại Cầu Giấy ngày 21-12-1873.
Thừa lúc Gác-ni-ê đem quân xuống đánh Nam Định, việc canh phòng Hà Nội sơ hở, quân ta do Hoàng Tá Viêm chỉ huy (có sự phối hợp với đội quân Cờ đen của Lưu Vĩnh Phúc) từ Sơn Tây kéo về Hà Nội, hình thành trận tuyến bao vây quân địch. Nghe tin đó, Gác-ni-ê phải tức tốc đưa quân vào sát thành Hà Nội khiêu chiến. Gác-ni-ê de quân đuổi theo. Rơi vào ổ phục kích của quân ta tại khu vực Cầu Giấy, toán quân Pháp, trong đó có Gác-ni-ê, đã bị tiêu diệt.
Chiến thắng Cầu Giấy lần thứ nhất khiến cho nhân dân ta vô cùng phấn khởi; ngược lại, làm cho thực dân Pháp hoang mang, lo sợ và tìm cách thương lương. Triều đình lại kí Hiệp ước năm 1874 (Hiệp ước Giáp Tuất), theo đó quân Pháp rút khỏi Hà Nội và các tỉnh đồng bằng Bắc Kì, nhưng vẫn có điều kiện tiếp tục xây dựng cơ sở để thực hiện các bước xâm lược về sau.
"""
#ground truth = "The output format of the tasks that the Florence-2 model can handle is text forms, whether it be captioning, object detection, grounding or segmentation."
     

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
     

In [ ]:
#new_model.device

In [ ]:

outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=200,
                        #  do_sample=True,
                        #  top_p=0.9,
                         temperature=0.6)
     

In [ ]:
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:

print(result)

In [ ]:

# Merge LoRA and base model
merged_model = new_model.merge_and_unload()

In [ ]:
# Save the merged model
merged_model.save_pretrained("metallama2-7b-qa-tuned-merged", safe_serialization=True)
tokenizer.save_pretrained("metallama2-7b-qa-tuned-merged")

In [ ]:

# push merged model to the hub

hf_model_repo = "Katalan/VlspQA-llama2"
new_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)

In [5]:
import torch
from transformers import BitsAndBytesConfig

# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

hf_model_repo = "Katalan/VlspQA-llama2"

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)

# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo,
                                             quantization_config=bnb_config,
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ImportError: cannot import name 'inject_adapter_in_model' from 'peft' (/opt/conda/lib/python3.10/site-packages/peft/__init__.py)

In [ ]:
prompt = """
Trong một quần xã rừng tự nhiên ở vùng Đông Nam Á, các loài động vật ăn có cỡ lớn như bò rừng mỗi khi di chuyển thường đánh động và làm các loài côn trùng bay khỏi tổ. Lúc này, các loài chim như diệc bạc sẽ bắt các côn trùng bay khỏi tổ làm thức ăn. Việc côn trùng bay khỏi tổ cũng như việc chim diệc bạc bắt côn trùng không ảnh hưởng gì đến đời sống bò rừng. Chim gõ bò có thể bắt ve bét trên da bò rừng làm thức ăn. 
Mối quan hệ giữa diệc bạc và côn trùng dẫn đến hiện tượng gì?
"""

In [ ]:

# Generate response
%%time
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=200,
                         temperature=0.6)

result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Print the result
print(f"Generated response:\n{result}")

In [ ]:

import transformers

tokenizer = AutoTokenizer.from_pretrained("Katalan/VlspQA-llama2",  trust_remote_code=True)
pipeline = transformers.pipeline(
    "text-generation",
    model="Katalan/VlspQA-llama2",
    trust_remote_code=True

)
   

In [ ]:

%%time
sequences = pipeline(
    prompt,
    temperature=0.6,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

In [ ]:

for seq in sequences:
    print(seq['generated_text'])

In [ ]:
from huggingface_hub import HfApi

In [ ]:
api = HfApi()

In [ ]:
# api.upload_folder(
#     folder_path="/kaggle/working/llama2-7b-tuned-qna/checkpoint-500",
#     path_in_repo = ".",
#     repo_id="TrinhDacPhu/questionansweringllma2",
#     repo_type="model"
# )